# Get data from twitter

!pip install tweepy

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

# snscrape Playground

In [9]:
import snscrape.modules.twitter as sntwitter
import pandas
from datetime import datetime, timezone
import pandas as pd
import pytz

# snscrape GUI


In [ ]:
#import
import snscrape.modules.twitter as sntwitter
import pandas
from datetime import datetime, timezone
import pandas as pd
import pytz

In [17]:
#Define functions

def list_to_df(liste):
  # Create corpus.
  corpus = pd.DataFrame(liste, columns = ['date','id','tweet','user'])
  # Set Turkey timezone.
  eastern = pytz.timezone('Turkey')
  corpus['date'] = corpus['date'].dt.tz_convert(eastern)
  print("DONE: List converted to dataframe.")
  return corpus

def df_to_csv(corpus,file_name = 'result.csv'):
  corpus.to_csv(file_name, sep='æ', encoding='utf-8-sig', index = None)  # use æ unique operator to avoid confusion. 
  print("DONE: File written. File name ->  " + file_name) 

def set_language(lang):
  if lang == "":
    return lang
  return 'lang:'+lang + ' '
  
def get_date(d):
  d = str(d.strftime('%d/%m/%Y %H:%M'))
  return d

def write_start_end_time(start_time, end_time):
  start_time = str(start_time.strftime('%d/%m/%Y %H:%M:%S'))
  end_time = str(end_time.strftime('%d/%m/%Y %H:%M:%S'))
  return start_time, end_time

def create_query(searched_word,excepted_word,replies, language, start_time,end_time):
  if(excepted_word == ""):
    pass
  else: 
    excepted_word_list = excepted_word.split(' ')
    excepted_word = ""
    for ex in excepted_word_list:
      excepted_word = excepted_word + "-" + ex +" "

  if replies == "n":
    replies = "-filter:replies"
  else: 
    replies = ""
  
  language = set_language(language)
  start_time = "since:"+start_time
  end_time = "until:"+end_time

  query = searched_word + " " + excepted_word + language + start_time + " " + end_time + " " + replies
  return query

def search_tweets(search_query, limitation,file_name):
  tweets = []
  start_time = datetime.now()
  counter = 1
  print("Tweet scraping started.")
  for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_query).get_items()):
    if limitation == 0:
      pass
    elif i > limitation: # Runtime is 2 mins for 5000 tweets. Try small number for test. 
      break
    counter += 1
    date = tweet.date
    date = get_date(date)
    if counter % 1000 == 0: print(counter) # 1 output per 50 tweets.
    tweets.append([tweet.date, tweet.id, tweet.content, tweet.username])

  print("Tweet scraping finished.")
  print("___________________________________________________________________")
  end_time = datetime.now()
  start_time, end_time = write_start_end_time(start_time,end_time)
  print('Start time: %25s\nEnd time: %29s\nTotal tweets: %7s'%(start_time,end_time,counter))
  if file_name == "":
    df_to_csv(list_to_df(tweets)) #df_to_csv(list_to_df(tweets),'yeni_csv_adi.csv')
  else: 
    file_name = file_name + '.csv'
    df_to_csv(list_to_df(tweets), file_name)
  print("___________________________________________________________________") 

def collect_tweets():
  searched_word = input("Word to search: ")
  excepted_word = input("Words to exclude (must have a space between words) : ")
  replies = input("Include answers? (default 'n') y/n?: ")
  language = input("For any language limitation, write it as shortening\nExample; for Turkish 'tr'\nfor English 'en'\nfor French 'fr'\nIf you don't want any limitation, leave empty: ")
  
  start_time = input("Start time? (YYYY-MM-DD 2022-01-01): ")
  end_time = input("End time? (YYYY-MM-DD 2022-01-31): ")
  limitation = int(input("Do you want any limitations? If you don't want it, write 0: "))
  return searched_word, excepted_word, replies, language, start_time, end_time, limitation

In [ ]:
#Collect Data#
word_to_search = ""
word_to_exclude = ""
replies_included = "n"
language = ""
start_time_date = ""
end_time_date = ""
search_query = ""
limitation = 0
#### Program Started ####
print("___________________________________________________________________")
word_to_search, word_to_exclude, replies_included, language, start_time_date, end_time_date, limitation = collect_tweets()
print("___________________________________________________________________")
search_query = create_query(word_to_search,word_to_exclude,replies_included,language, start_time_date, end_time_date)
print("Seacrh query:", search_query)
file_name = input("Use a specific filename? \nIf you leave it blank it will overwrite 'tweet_data' .\nFile name: ")
print("___________________________________________________________________")
#Finally searching part!
search_tweets(search_query, limitation,file_name)